In [2]:
%run "./1. Data Loading.ipynb"

X: (800, 360, 25)
Y: (800,)


In [6]:
from sklearn.model_selection import train_test_split

In [35]:
print(set(regions[1]))

{'Somatomotor', 'Auditory', 'Language', 'Frontopariet', 'Default', 'Posterior-Mu', 'Visual2', 'Orbito-Affec', 'Cingulo-Oper', 'Ventral-Mult', 'Dorsal-atten', 'Visual1'}


In [38]:
# Helper function:
def find_indices_matching_string(arr, target_string):
    indices = []
    for index, item in enumerate(arr):
        if item == target_string:
            indices.append(index)
    return indices

In [17]:
# Average related conditions in each run.
averaged_data = {}
for subject_index, subject_id in enumerate(manifest):
    averaged_data[subject_id] = []

    for run_index, run in enumerate(manifest[subject_id]):

        neut_block = []
        fear_block = []

        for block_index, block_lookup in enumerate(run["condition_spans"]): 

            if (block_index+1 == 6):
                break
            
            start_index = block_lookup['Frames'][0]
            end_index = block_lookup['Frames'][1]
            condition = 0 if block_lookup['Condition'] == "Neut" else 1
            
            block_data = run["data"][:, start_index : end_index]

            if (condition == 0):
                neut_block.append(block_data)
            elif (condition == 1):
                fear_block.append(block_data)

        neut = np.mean(neut_block, axis=0)
        fear = np.mean(fear_block, axis=0)

        averaged_data[subject_id].append({
            "neut": neut,
            "fear": fear
        })

print(averaged_data["100307"][0]["neut"].shape)
print(averaged_data["100307"][0]["fear"].shape)

(360, 25)
(360, 25)


In [47]:
# Generate correlation matrix for our seed region against a target network.
seed_region = "L_V1"
seed_index = list(regions[0]).index(seed_region)

target_network = "Default"
target_network_indices = find_indices_matching_string(regions[1], target_network)

# NOTE: seed_region must not be in target_region or logic error. 
if regions[1][seed_index] == target_network:
    quit()

for subject_index, subject_id in enumerate(averaged_data):
    for run_index, run in enumerate(averaged_data[subject_id]):
        neut_data = run["neut"]
        fear_data = run["fear"]

        neut_correlation_matrix = np.corrcoef(neut_data)
        fear_correlation_matrix = np.corrcoef(fear_data)

        neut_seed_matrix = neut_correlation_matrix[seed_index, target_network_indices]
        fear_seed_matrix = fear_correlation_matrix[seed_index, target_network_indices]

        averaged_data[subject_id][run_index]["neut_seed_matrix"] = neut_seed_matrix
        averaged_data[subject_id][run_index]["fear_seed_matrix"] = fear_seed_matrix

In [ ]:
# Split train & test. 